In [ ]:
import numpy as np
import pandas as pd

from ptplot import plotting, utilities

%reload_ext autoreload
%autoreload 2

We'll pick a single play to look at, just for demonstration purposes. I've somewhat-randomly chosen [this Cleveland Browns reverse + WR pass from the 2018 season](https://www.youtube.com/watch?v=Co5IaAFdaPU), which has player-tracking data available for it from Adam Sonty's [ngs_highlights repo](https://github.com/asonty/ngs_highlights) (specifically, the file used is located [here](https://raw.githubusercontent.com/asonty/ngs_highlights/20f74138f0a2f7e91879d31bbf2d8addc0bd59bc/play_data/2018_CLE_2018122305_1246.tsv)).

In [ ]:
# Data file copied locally to ensure the demo works even if the ngs_highlights_repo is taken down
player_tracking_data = pd.read_csv(
    "2018_CLE_2018122305_1246.tsv",
    sep="\t", parse_dates=["time"]
)

In [ ]:
player_tracking_data.head().T

Let's start by making a simple plot that shows player positions at a certain frame. Frames where interesting stuff happens are actually tagged by the event column, so we'll choose the moment of time when the pass occurred:

In [ ]:
fig = plotting.plot_positions(
    player_tracking_data[player_tracking_data["event"] == "pass_forward"], 
    "x", "y"
)
fig.show()

That's nice, but all the players look the same, which isn't very helpful. The good news is that it's pretty easy to make things clearer. First we'll add a column to the DataFrame to denote which rows contain the ball, then we'll use that column + others to add team and player information:

In [ ]:
player_tracking_data["is_ball"] = (player_tracking_data["displayName"] == "ball")

In [ ]:
fig = plotting.plot_positions(
    player_tracking_data[player_tracking_data["event"] == "pass_forward"], "x", "y",
    uniform_number="jerseyNumber",
    home_away_identifier="homeTeamFlag",
    hover_text="displayName",
    ball_identifier="is_ball",
    team_abbreviations="teamAbbr"
)
fig.show()

Much better. In addition to the addition of team colors and player numbers, note that mousing over the markers shows the player's name too. You can also use the Plotly controls on the top right of the chart to do things like zoom in, select players, and save a static image. 

in addition to making plots of player positions, `ptplot` can also generate tracks of player positions over the course of the play. The code in the next cell generates those tracks for the entire play:

In [ ]:
fig = plotting.plot_tracks(
    player_tracking_data, "x", "y", "displayName",
    home_away_identifier="homeTeamFlag",
    hover_text="displayName",
    ball_identifier="is_ball",
    team_abbreviations="teamAbbr"
)
fig.show()

Again you'll see similar interactivity as for the player positions plot. 

Now, if you want both the positions *and* player tracks, it's possible to combine these two figures by re-using the figure object that `ptplot`'s plotting functions return:

In [ ]:
# Just look between the snap and the pass, to make the tracks clearer
snap_frame = player_tracking_data[player_tracking_data["event"] == "ball_snap"]["frame"].unique()[0]
pass_forward_frame = player_tracking_data[player_tracking_data["event"] == "pass_forward"]["frame"].unique()[0]

fig = plotting.plot_tracks(
    player_tracking_data[player_tracking_data["frame"].between(snap_frame, pass_forward_frame)], 
    "x", "y", "displayName",
    home_away_identifier="homeTeamFlag",
    hover_text="displayName",
    ball_identifier="is_ball",
    team_abbreviations="teamAbbr"
)
plotting.plot_positions(
    player_tracking_data[player_tracking_data["event"] == "pass_forward"], "x", "y",
    uniform_number="jerseyNumber",
    home_away_identifier="homeTeamFlag",
    hover_text="displayName",
    ball_identifier="is_ball",
    team_abbreviations="teamAbbr",
    fig=fig
)
fig.show()

Lastly, `ptplot` comes with support for both horizontal and vertical field orientations. Horizontal views tend to be better for computers, which tend towards widescreen displays, while vertical views scroll better on phones.

In [ ]:
# Same plot as before, but now vertically oriented
vertical_fig = plotting.plot_tracks(
    player_tracking_data[player_tracking_data["frame"].between(snap_frame, pass_forward_frame)], 
    "y", "x", "displayName",
    home_away_identifier="homeTeamFlag",
    hover_text="displayName",
    ball_identifier="is_ball",
    team_abbreviations="teamAbbr",
    fig="nfl_vertical"
)
plotting.plot_positions(
    player_tracking_data[player_tracking_data["event"] == "pass_forward"], "y", "x",
    uniform_number="jerseyNumber",
    home_away_identifier="homeTeamFlag",
    hover_text="displayName",
    ball_identifier="is_ball",
    team_abbreviations="teamAbbr",
    fig=vertical_fig
)
vertical_fig.update_layout(width=250, height=550)
vertical_fig.show()

In [ ]:
positions_fig = plotting.animate_play(
    player_tracking_data, "x", "y", "frame",
    uniform_number="jerseyNumber",
    home_away_identifier="homeTeamFlag",
    hover_text=lambda data: data["displayName"].str.cat(data["position"], sep=" "),
    ball_identifier=lambda data: (data["displayName"] == "ball").values,
    #slider_label_generator=utilities.generate_time_elapsed_labels(player_tracking_data["time"].min(), "time"),
    team_abbreviations="teamAbbr",
    events_of_interest="event"
)
positions_fig.show()

In [ ]:
# frames_to_track = (
#     (player_tracking_data["frame"] < 100)
# )
tracks_fig = plotting.animate_tracks(
    player_tracking_data[["x", "y", "displayName", "frame"]], "x", "y", "displayName", "frame",
    home_away_identifier="homeTeamFlag",
    hover_text=lambda data: data["displayName"].str.cat(data["position"].fillna(""), sep=" "),
    ball_identifier=lambda data: (data["displayName"] == "ball").values,
    team_abbreviations="teamAbbr",
    events_of_interest="event"
)
tracks_fig.show()

In [ ]:
player_tracking_data[["x", "y", "displayName", "frame"]].memory_usage().sum() / 1e6

In [ ]:
player_tracking_data

In [ ]:
combined_fig = plotting.animate_play(
    player_tracking_data, "x", "y", "frame",
    uniform_number="jerseyNumber",
    home_away_identifier="homeTeamFlag",
    hover_text=lambda data: data["displayName"].str.cat(data["position"], sep=" "),
    ball_identifier=lambda data: (data["displayName"] == "ball").values,
    team_abbreviations="teamAbbr",
    events_of_interest="event",
    fig=tracks_fig
)
combined_fig.show()

In [ ]:
fig = plotting.plot_positions(
    player_tracking_data[player_tracking_data["event"] == "pass_forward"], "y", "x",
    uniform_number="jerseyNumber",
    home_away_identifier="homeTeamFlag",
    hover_text=lambda data: data["displayName"].str.cat(data["position"], sep=" "),
    ball_identifier=lambda data: (data["displayName"] == "ball").values,
    team_abbreviations="teamAbbr",
    fig="nfl_vertical"
)
fig.update_layout(width=400, height=700)
fig.show()